## Data Preparation

0. Required library and Import the Data

In [44]:
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn.datasets import load_diabetes
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
pd.set_option("display.max_colwidth", 500)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from datetime import datetime, timedelta
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
df = pd.read_csv('./bangkok_traffy.csv')
df.drop_duplicates(inplace=True)

- Make 'type' as list

In [46]:
df['type'] = (
    df['type']
    .str.strip("{}")          # remove leading/trailing {}
    .str.split(",")           # split into list
)


In [47]:
df.loc[[289638,196012,142463], ['ticket_id','type','organization','comment','address','subdistrict']]

,ticket_id,type,organization,comment,address,subdistrict
289638,2023-7ZTRF9,"[สะพาน, ท่อระบายน้ำ, ถนน, ร้องเรียน, ความปลอดภัย]","เขตหนองแขม,ฝ่ายโยธา เขตหนองแขม",[ ทีมงานว่าที่ สส. รักชนก ] ได้รับเรื่องร้องเรียน เรื่อง เรียบถนนเส้นข้างโรงเรียนวัดหนองแขมทางยาวไปถึงสะพานไม้ มีท่อระบายน้ำเป็นหลุม ฝาท่อสูงเกินกว่าถนน ทำให้เกิดอันตราย อยากให้เอายางมะตอยมาราดเพื่อให้ถนนเสมอกันเรียบสูง,91 ถนนเลียบคลองภาษีเจริญฝั่งใต้ แขวงหนองแขม เขตหนองแขม กรุงเทพมหานคร 10160 ประเทศไทย,หนองแขม
196012,2023-TWGYXL,[],"เขตบางซื่อ,ฝ่ายเทศกิจ เขตบางซื่อ,ผอ.เขตบางซื่อ (นายเจษฎา)",ป้ายโฆษณาครับ,22 ถ. รัชดาภิเษก แขวง วงศ์สว่าง เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,วงศ์สว่าง
142463,2022-7QUGYN,[ถนน],"เขตหลักสี่,ผอ.เขตหลักสี่ (นางสมฤดี),กลุ่มกรุงเทพเหนือ (นางวันทนีย์ วัฒนะ)",ตรงซอยถนนหน้าสำนักงานเขตหลักสี่,117/6 แจ้งวัฒนะ แขวง ทุ่งสองห้อง เขต หลักสี่ กรุงเทพมหานคร 10210 ประเทศไทย,ทุ่งสองห้อง


- set ticker_id as index

In [48]:
df['type'].apply(type).unique()
df.set_index('ticket_id', inplace=True)



In [49]:
df.loc[['2023-7ZTRF9', '2023-TWGYXL']]


,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
ticket_id,,,,,,,,,,,,,,,
2023-7ZTRF9,"[สะพาน, ท่อระบายน้ำ, ถนน, ร้องเรียน, ความปลอดภัย]","เขตหนองแขม,ฝ่ายโยธา เขตหนองแขม",[ ทีมงานว่าที่ สส. รักชนก ] ได้รับเรื่องร้องเรียน เรื่อง เรียบถนนเส้นข้างโรงเรียนวัดหนองแขมทางยาวไปถึงสะพานไม้ มีท่อระบายน้ำเป็นหลุม ฝาท่อสูงเกินกว่าถนน ทำให้เกิดอันตราย อยากให้เอายางมะตอยมาราดเพื่อให้ถนนเสมอกันเรียบสูง,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-06/5a65cb9be7e1a69cd838e9e30812aa4bbaf0ef4c.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-06/1686119531922.jpeg,"100.33999,13.67570",91 ถนนเลียบคลองภาษีเจริญฝั่งใต้ แขวงหนองแขม เขตหนองแขม กรุงเทพมหานคร 10160 ประเทศไทย,หนองแขม,หนองแขม,กรุงเทพมหานคร,2023-06-06 08:08:14.457214+00,เสร็จสิ้น,4.0,0,2023-06-07 06:32:13.130699+00
2023-TWGYXL,[],"เขตบางซื่อ,ฝ่ายเทศกิจ เขตบางซื่อ,ผอ.เขตบางซื่อ (นายเจษฎา)",ป้ายโฆษณาครับ,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-01/5680383cc9125134b3102adb397d0d7f17360059.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-01/8b9b18cc01db9e81708ac8c5e54cafa3eb5a9a33.jpg,"100.53265,13.83065",22 ถ. รัชดาภิเษก แขวง วงศ์สว่าง เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,วงศ์สว่าง,บางซื่อ,กรุงเทพมหานคร,2023-01-11 15:33:43.676854+00,เสร็จสิ้น,NaN,0,2023-01-12 05:11:10.739687+00


In [50]:
[k in df.index for k in ['2023-7ZTRF9', '2023-TWGYXL']]
df.shape


(787026, 15)

In [51]:
df.loc['2023-7ZTRF9']

type                                                                                                                                                                                       [สะพาน, ท่อระบายน้ำ, ถนน, ร้องเรียน, ความปลอดภัย]
organization                                                                                                                                                                                                  เขตหนองแขม,ฝ่ายโยธา เขตหนองแขม
comment          [ ทีมงานว่าที่ สส. รักชนก ] ได้รับเรื่องร้องเรียน เรื่อง เรียบถนนเส้นข้างโรงเรียนวัดหนองแขมทางยาวไปถึงสะพานไม้ มีท่อระบายน้ำเป็นหลุม ฝาท่อสูงเกินกว่าถนน ทำให้เกิดอันตราย อยากให้เอายางมะตอยมาราดเพื่อให้ถนนเสมอกันเรียบสูง
photo                                                                                                                    https://storage.googleapis.com/traffy_public_bucket/attachment/2023-06/5a65cb9be7e1a69cd838e9e30812aa4bbaf0ef4c.jpg
photo_after                                         

- Drop column that won't be used

In [52]:
df = df[df['province'].str.contains('กรุงเทพ', na=False)]
df.drop(columns=['photo','photo_after','coords','province','star','state','count_reopen','last_activity'],inplace=True)
df.shape

(785656, 7)

In [53]:
df.iloc[1]['address']
print(df.loc['2024-4B6VD4'].address)

704 ถ. ประชาธิปก แขวงสมเด็จเจ้าพระยา เขตคลองสาน กรุงเทพมหานคร 10600 ประเทศไทย


In [54]:
df.iloc[2]

type                                                                          [สะพาน]
organization                                                                  เขตสาทร
comment                               สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12
address         191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย
subdistrict                                                                   ยานนาวา
district                                                                         สาทร
timestamp                                               2021-09-26 05:03:52.594898+00
Name: 2021-7XATFA, dtype: object

- Extract 'district' and 'subdistrict' from 'address'

In [55]:
def extract_subdistrict(address):
    if not isinstance(address, str):
        return None
    match = re.search(r'แขวง\s*([^\s]+)', address)
    if match:
        return match.group(1)
    return None

def extract_district(address):
    if not isinstance(address, str):
        return None
    match = re.search(r'เขต\s*([^\s]+)', address)
    if match:
        return match.group(1)
    return None
df=df.copy()

extracted_sub = df['address'].apply(extract_subdistrict)
extracted_dist = df['address'].apply(extract_district)

df['subdistrict'] = df['subdistrict'].fillna(extracted_sub)
df['district'] = df['district'].fillna(extracted_dist)

In [56]:
df['district'] = df['district'].apply(lambda x: np.nan if x in [None, [], ''] else x)
df['subdistrict'] = df['subdistrict'].apply(lambda x: np.nan if x in [None, [], ''] else x)
df.dropna(subset=['district','subdistrict'],inplace=True)
print(df.district.unique().tolist())

['บางซื่อ', 'ประเวศ', 'สาทร', 'ลาดพร้าว', 'ดุสิต', 'สายไหม', 'ราชเทวี', 'บางรัก', 'จตุจักร', 'บางแค', 'บางกอกน้อย', 'คลองเตย', 'ลาดกระบัง', 'หลักสี่', 'บางกะปิ', 'หนองจอก', 'วัฒนา', 'บางเขน', 'คลองสาน', 'ราษฎร์บูรณะ', 'ดินแดง', 'ธนบุรี', 'ปทุมวัน', 'คันนายาว', 'พระนคร', 'ตลิ่งชัน', 'ดอนเมือง', 'ห้วยขวาง', 'สะพานสูง', 'พญาไท', 'สวนหลวง', 'ภาษีเจริญ', 'บางขุนเทียน', 'บางคอแหลม', 'บางนา', 'ทุ่งครุ', 'จอมทอง', 'คลองสามวา', 'วังทองหลาง', 'บางกอกใหญ่', 'ยานนาวา', 'พระโขนง', 'บึงกุ่ม', 'หนองแขม', 'มีนบุรี', 'บางพลัด', 'บางบอน', 'สัมพันธวงศ์', 'ป้อมปราบศัตรูพ่าย', 'ทวีวัฒนา', 'บางกรวย', 'บางพลี', 'เมืองนนทบุรี', 'พระประแดง', 'ลำลูกกา', 'เมืองสมุทรสาคร', 'สามพราน', 'ป้อมปราบศัตรูพ่า', 'เมืองปทุมธานี']


In [57]:
valid_districts = [
    "พระนคร", "ดุสิต", "หนองจอก", "บางรัก", "บางเขน", "บางกะปิ",
    "ปทุมวัน", "ป้อมปราบศัตรูพ่าย", "พระโขนง", "มีนบุรี", "ลาดกระบัง",
    "ยานนาวา", "สัมพันธวงศ์", "พญาไท", "ธนบุรี", "บางกอกใหญ่",
    "ห้วยขวาง", "คลองสาน", "ตลิ่งชัน", "บางกอกน้อย", "บางขุนเทียน",
    "ภาษีเจริญ", "หนองแขม", "ราษฎร์บูรณะ", "บางพลัด", "ดินแดง",
    "บึงกุ่ม", "สาทร", "บางซื่อ", "จตุจักร", "บางคอแหลม", "ประเวศ",
    "คลองเตย", "สวนหลวง", "จอมทอง", "ดอนเมือง", "ราชเทวี",
    "ลาดพร้าว", "วัฒนา", "บางแค", "หลักสี่", "สายไหม", "คันนายาว",
    "สะพานสูง", "วังทองหลาง", "คลองสามวา", "บางนา", "ทวีวัฒนา",
    "ทุ่งครุ", "บางบอน"
]
df['district'] = df['district'].str.strip()
df = df[df['district'].isin(valid_districts)]

In [58]:
df.sample(3)

,type,organization,comment,address,subdistrict,district,timestamp
ticket_id,,,,,,,
2024-GWZZ3C,[สะพาน],"เขตบางคอแหลม,องค์การขนส่งมวลชนกรุงเทพ (ขสมก.)",ป้ายรถเมล์ตรงนี้อ่ะคะ พระราม 3 ซอย 6 สาย 205 มันไม่ค่อยจอดเลย เรียกแล้วก็ไม่จอด กระเป๋าบอกว่ารถเมล์เข้ามาจอดไม่ได้ เข้ามาจอดไม่ทัน แล้วต้องขึ้นสะพานกรุงเทพขึ้นไม่ได้ มานั่งรอแต่รถไม่แวะรับเสียเวลามากค่ะ,10/7 ถ. พระรามที่ 3 แขวงบางคอแหลม เขตบางคอแหลม กรุงเทพมหานคร 10120 ประเทศไทย,บางคอแหลม,บางคอแหลม,2024-01-29 08:14:55.998742+00
2024-LC2RLN,"[ความปลอดภัย, ทางเท้า]","เขตบางรัก,ฝ่ายโยธา เขตบางรัก",น้ำจากท่อระบาย ของร้านค้าไหลลงมาตรงทางเดินสาธารณะ \nทำให้เกิดอุบัติเหตุไปแล้วนะครับ และส่งผลเสียให้กับร้านค้าในระแวกนั้น แจ้งไปตั้งแต่\n วันที่9 เดือน8 ก็ยังไม่ได้รับการแก้ไข,1310 ถ. เจริญกรุง แขวงสุริยวงศ์ เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,บางรัก,บางรัก,2024-10-01 10:38:51.702687+00
TTBV9Z,"[ถนน, สัตว์จรจัด, ร้องเรียน]","เขตบางขุนเทียน,ฝ่ายสิ่งแวดล้อมฯ เขตบางขุนเทียน",ปัญหา: อ้างอิงเลขที่ร้องทุกข์ NGV4HH สถานะ เสร็จสิ้น\nเรื่อง บ้านหลังดังกล่าว ปล่อยแมวเข้ามายังบ้านผู้แจ้ง บ้านเลขที่ 9 (บ้านผู้ก่อเหตุ)\nเจ้าหน้าที่ตอบกลับมาเมื่อ ศ. 15 ธ.ค. 66 12:55\nสำนักงานเขตบางขุนเทียนตรวจสอบแล้วเป็นบ้านพักอาศัยซอยเทียนทะเล20 ขณะตรวจสอบไม่พบว่ามีแมวออกมาจากบ้านเลขที่9ซอยดังกล่าว\n\nปัจจุบันผู้ร้องแจ้งว่า ปัญหายังไม่ได้รับการแก้ไข เนื่องจากเจ้าหน้าที่เข้ามาตรวจสอบแล้ว หลังจากนั้นพบว่าบ้านหลังดังกล่าวยังมีพฤติกรรมเช่นเดิม\nบ้านเลขที่: 9\nซอย: เทียนทะเล 20 แยก 8-5-6-5\nถน...,8 ซ. เทียนทะเล 20 แยก 8-5-6-5 แขวงแสมดำ เขตบางขุนเทียน กรุงเทพมหานคร 10150 ประเทศไทย,แสมดำ,บางขุนเทียน,2023-12-17 05:28:20.538722+00


In [59]:
df.timestamp = pd.to_datetime(df.timestamp, format='mixed')
df['timestamp_thai'] = df.timestamp.dt.tz_convert('Asia/Bangkok')

df['year'] = df['timestamp_thai'].dt.year
df['month'] = df['timestamp_thai'].dt.month
df['day'] = df['timestamp_thai'].dt.day

df.drop(columns=['timestamp','timestamp_thai'],inplace=True)

In [60]:
df.sample(3)

,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
2022-7GQQZN,[กีดขวาง],"เขตบางรัก,สำนักการโยธา กทม.,สำนักงานก่อสร้างและบูรณะ สำนักการโยธา (สกบ.สนย.) กทม.,ศูนย์ก่อสร้างและบูรณะถนน 2 ส่วนก่อสร้างและบูรณะ 1 สำนักงานก่อสร้างและบูรณะ (สกบ.สนย.)",ขวางทางเดิน,สีลม แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,สีลม,บางรัก,2022,12,16
2024-ECMC4X,"[ถนน, ความปลอดภัย]","เขตหนองจอก,สำนักการโยธา กทม.,ศูนย์ก่อสร้างและบูรณะถนน 1 ส่วนก่อสร้างและบูรณะ 1 สำนักงานก่อสร้างและบูรณะ (สกบ.สนย.)",อยากให้พิจารณาทำคันชะลอความเร็ว เพราะทำถนนใหม่เสร็จเรียบร้อยแล้วรถยิ่งวิ่งเร็วมาก จะทำให้เกิดอุบัติเหตุ,79 ซอย ฉลองกรุง 53 แขวงลำปลาทิว เขตลาดกระบัง กรุงเทพมหานคร 10520 ประเทศไทย,ลำผักชี,หนองจอก,2024,6,6
26ZD8P,"[ถนน, แสงสว่าง]","เขตมีนบุรี,กรมทางหลวง",//กรุณากรุณาข้อมูลบรรยายประกอบกับปักหมุด//\nปัญหาที่แจ้ง: ไฟส่องสว่างถนนร่มเกล้า จากมีนบุรีไปลาดกระบังถนนหมายเลข 3119 ดับเป็นระยะ เป็นช่วงๆ ช่วยซ่อมเปลี่ยนด้วยครับ\nจุดเกิดเหตุ: ถนนร่มเกล้า เขตมีนบุรีไปลาดกระบัง \nจุดสังเกต: -\nถนน: ร่มเกล้า\nซอย:-\n เขตมีนบุรี กรุงเทพมหานคร 10510\r\n#1555,- ร่มเกล้า มีนบุรี มีนบุรี กรุงเทพมหานคร ถนนร่มเกล้า เขตมีนบุรีไปลาดกระบังหมายเลข 3119 : RP9M+JPM แยกร่มเกล้า-สุวินทวงศ์ เขตมีนบุรี กรุงเทพมหานคร 10510 ประเทศไทย,มีนบุรี,มีนบุรี,2023,3,7


In [61]:
df.columns.tolist()

['type',
 'organization',
 'comment',
 'address',
 'subdistrict',
 'district',
 'year',
 'month',
 'day']

In [62]:
df

,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
2021-FYJTFP,[ความสะอาด],เขตบางซื่อ,ขยะเยอะ,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,9,3
2021-CGPMUN,"[น้ำท่วม, ร้องเรียน]","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,2021,9,19
2021-7XATFA,[สะพาน],เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,ยานนาวา,สาทร,2021,9,26
2021-9U2NJT,[น้ำท่วม],"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,10,14
2021-DVEWYM,"[น้ำท่วม, ถนน]","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว\n\n1. หลังจากฝนตกต่อเนื่องประมาณ 30 นาที น้ำท่วมซอยสูงประมาณ 20 ซม ใช้เวลาระบายราวๆ 3-5 ชม. กว่าน้ำในซอยแห้ง\nหากฝนตกนานกว่า 1 ชม ใช้เวลา 2-3 วัน กว่าน้ำในซอยแห้ง\nหากนานกว่านั้น ต้นซอยน้ำแห้ง ปลายซอยน้ำเน่า,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว กรุงเทพมหานคร 10230 ประเทศไทย,ลาดพร้าว,ลาดพร้าว,2021,12,9
...,...,...,...,...,...,...,...,...,...
KDDNBP,[ร้องเรียน],เขตหนองจอก,ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบบการจัดการเรื่องราวร้องทุกข์ สำนักงานปลัดสำนักนายกรัฐมนตรี ทำเนียบรัฐบาล รหัส นร02680001844\n\nขอให้ตรวจสอบการจำหน่ายเครื่องดื่มแอลกอฮอล์ก่อนเวลาที่กฎหมายกำหนด บริเวณ ร้านจำหน่ายสินค้าเบ็ดเตล็ด ชื่อ .... (ไม่ทราบเลขที่ตั้ง) การเคหะหนองจอก ซอย 11ก แขวงคู้ฝั่งเหนือ เขตหนองจอก กรุงเทพมหานคร จุดสังเกต ร้านจะอยู่ด้านหน้าการเคหะหนองจอก ปากซอย 11ก ลักษณะเป็นร้านปูนสีขาว ชั้นเดียว เนื่องจากผู้ร้องแจ้งว่า ร้านดังกล่าวมีเจ้าของร้านเป็นบุคคลเพศชาย (ทราบชื่...,เขตหนองจอก กรุงเทพมหานคร : VV47+G6M แขวงกระทุ่มราย เขตหนองจอก กรุงเทพมหานคร 10530 ประเทศไทย,กระทุ่มราย,หนองจอก,2025,1,16
RYM99G,"[ความสะอาด, ถนน]","เขตคลองสาน,ฝ่ายสิ่งแวดล้อมฯ เขตคลองสาน",ปัญหา: บริเวณตลาดพระเครื่องพญาไม้ พบร้านอาหารตามสั่งและร้านก๋วยเตี๋ยว ประกอบอาหารส่งกลิ่นเหม็นและมีควันรบกวน ตั้งแต่เวลา 08.00-12.00 น.เป็นประจำทุกวัน ยกเว้นวันอาทิตย์ สร้างความเดือดร้อนให้กับผู้พักอาศัยบริเวณใกล้เคียง\nถนน: พญาไม้\nจุดสังเกต: อยู่บริเวณกลางตลาด\nเขต: คลองสาน\r\n#1555,39 ถ. สมเด็จเจ้าพระยา แขวงสมเด็จเจ้าพระยา เขตคลองสาน กรุงเทพมหานคร 10600 ประเทศไทย,สมเด็จเจ้าพระยา,คลองสาน,2025,1,16
2025-BDQ87Y,"[ร้องเรียน, ต้นไม้, ความปลอดภัย]",เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16


In [63]:
df[df.type.isna()].shape[0]

97

In [64]:
df = df[df['type'].notna()]

In [65]:
# Convert the column to string format momentarily to check unique values
df['type'].astype(str).value_counts()

# Now print it

type
['']                                                         114965
['ถนน']                                                       85461
['ทางเท้า']                                                   54431
['แสงสว่าง']                                                  34122
['ความสะอาด']                                                 32182
                                                              ...  
['เสียงรบกวน', 'ถนน', 'จราจร', 'สะพาน', 'เสนอแนะ']                1
['สายไฟ', 'การเดินทาง', 'ต้นไม้']                                 1
['ทางเท้า', 'ป้าย', 'ถนน', 'ความสะอาด', 'สัตว์จรจัด']             1
['ความปลอดภัย', 'ทางเท้า', 'กีดขวาง', 'ถนน', 'ร้องเรียน']         1
['คลอง', 'ร้องเรียน', 'แสงสว่าง', 'ทางเท้า']                      1
Name: count, Length: 17443, dtype: int64

In [66]:
# 1. Explode the list column to separate the tags
all_individual_tags = df.explode('type')

# 2. Count the frequency of every single tag
# This gives you the "Master List" of categories you can use
tag_counts = all_individual_tags['type'].value_counts()

print("--- Top 20 Tags Found in Data ---")
print(tag_counts)

# Get the list of names for copy-pasting
unique_tags_list = tag_counts.index.tolist()


--- Top 20 Tags Found in Data ---
type
ถนน            255725
               114965
ทางเท้า        108748
ความปลอดภัย     81525
แสงสว่าง        72847
ความสะอาด       67451
กีดขวาง         66904
ร้องเรียน       50784
จราจร           46909
ท่อระบายน้ำ     42670
น้ำท่วม         41744
ป้าย            39079
เสียงรบกวน      35346
สะพาน           34403
ต้นไม้          29592
คลอง            25796
สายไฟ           24783
สัตว์จรจัด      16505
คนจรจัด          7414
PM2.5            7385
สอบถาม           6722
เสนอแนะ          5322
การเดินทาง       2337
ห้องน้ำ          1894
ป้ายจราจร        1422
Name: count, dtype: int64


In [67]:


# 1. Define the Keyword Map
# Dictionary Format: { 'Tag Name': [List of keywords that imply this tag] }
keyword_map = {
    'ถนน': ['หลุม', 'บ่อ', 'ยางมะตอย', 'ทรุด', 'พัง', 'ผิวจราจร', 'road', 'surface'],
    'ทางเท้า': ['ฟุตบาท', 'ทางเดิน', 'กระเบื้อง', 'sidewalk', 'footpath', 'อิฐ'],
    'แสงสว่าง': ['ไฟ', 'มืด', 'ดับ', 'ส่องสว่าง', 'light', 'lamp'],
    'ความสะอาด': ['ขยะ', 'สกปรก', 'เหม็น', 'รก', 'dirty', 'trash', 'rubbish'],
    'กีดขวาง': ['จอด', 'ขวาง', 'ร้านค้า', 'แผงลอย', 'block'],
    'ท่อระบายน้ำ': ['ท่อ', 'ฝา', 'ระบาย', 'drain'],
    'น้ำท่วม': ['ท่วม', 'ขัง', 'flood', 'water'],
    'จราจร': ['รถติด', 'รถมาก', 'traffic', 'jam'],
    'สายไฟ': ['สาย', 'ห้อย', 'ระโยง', 'cable', 'wire'],
    'ต้นไม้': ['ต้นไม้', 'กิ่ง', 'ตัด', 'หญ้า', 'tree', 'branch'],
    'สัตว์จรจัด': ['หมา', 'แมว', 'สุนัข', 'กัด', 'เห่า', 'dog', 'cat'],
    'เสียงรบกวน': ['เสียง', 'ดัง', 'เพลง', 'noise', 'loud'],
    'คลอง': ['คลอง', 'น้ำเสีย', 'canal'],
    'PM2.5': ['ฝุ่น', 'ควัน', 'dust', 'pm', 'อากาศ'],
    'ความปลอดภัย': ['อันตราย', 'โจร', 'ขโมย', 'เปลี่ยว', 'safe', 'crime'],
    'ป้าย': ['ป้าย', 'sign'],
    'สะพาน': ['สะพาน', 'bridge', 'overpass']
}

# 2. Define the Rescue Function
def recover_type_smart(row):
    # Get current values
    current_type = row['type']
    comment = str(row['comment']) # Convert to string to avoid errors

    # CHECK: Is the type currently empty?
    # We check for: empty list [], list with empty string [''], or None
    is_empty = False
    if isinstance(current_type, list):
        if len(current_type) == 0: is_empty = True
        elif len(current_type) == 1 and current_type[0] == '': is_empty = True
    elif pd.isna(current_type):
        is_empty = True

    # IF VALID: Return it as is (Don't change existing data)
    if not is_empty:
        return current_type

    # IF EMPTY: Try to guess based on keywords
    found_tags = set() # Use a set to avoid duplicates

    for tag, keywords in keyword_map.items():
        for word in keywords:
            if word in comment:
                found_tags.add(tag)
                # Once we match a keyword for this tag, stop checking other keywords for the same tag
                break

    # Return result
    if len(found_tags) > 0:
        return list(found_tags) # Convert back to list
    else:
        return ['Unspecified'] # Final fallback if no keywords match

# 3. Apply the function
# We create a NEW column first to be safe
df['type'] = df.apply(recover_type_smart, axis=1)

# 4. Check the Improvement
# Count how many are still 'Unspecified' vs how many we recovered
print("--- After Recovery ---")
# Explode again to count the new tags
print(df['type'].explode().value_counts().head(20))


--- After Recovery ---
type
ถนน            261793
ทางเท้า        111121
แสงสว่าง        85610
ความปลอดภัย     82964
กีดขวาง         80266
ความสะอาด       76958
Unspecified     54485
ร้องเรียน       50784
ท่อระบายน้ำ     48136
จราจร           47228
ป้าย            46350
น้ำท่วม         42950
เสียงรบกวน      40052
สะพาน           35539
ต้นไม้          33126
สายไฟ           28635
คลอง            27873
สัตว์จรจัด      22352
PM2.5           12009
คนจรจัด          7414
Name: count, dtype: int64


C:\Users\Predator\AppData\Local\Temp\ipykernel_23680\2843960521.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'] = df.apply(recover_type_smart, axis=1)


In [68]:
target_tags = {'สัตว์จรจัด', 'ถนน', 'คนจรจัด'} # Use {} for a set

# Filter rows where the set of tags matches exactly
# (We use isinstance(x, list) to prevent errors if there are NaNs)
df[df['type'].apply(lambda x: set(x) == {'สัตว์จรจัด', 'ถนน', 'คนจรจัด'} if isinstance(x, list) else False)]


,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
PTVRDY,"[ถนน, สัตว์จรจัด, คนจรจัด]","เขตธนบุรี,ฝ่ายสิ่งแวดล้อมฯ เขตธนบุรี",* ปัญหา: สุนัขจรจัดเข้ามาไล่กัดคน\n* จุดเกิดเหตุ: ภายในซอยวัดกันตทาราราม\n* บ้านเลขที่:-\n* ซอย: ซอยวัดกันตทาราราม\n* ถนน: ถนนราชพฤกษ์\n* เขต: เขตธนบุรี\n\nประชาชนแจ้งว่า พบสุนัขจรจัดเข้ามาไล่กัดคนบริเวณภายในซอยดังกล่าวคาดว่าน่าจะเป็นสุนัขของคนเร่ร่อนภายในซอย\r\n#1555,86/21 ซอย เทอดไท 19 แขวงตลาดพลู เขตธนบุรี กรุงเทพมหานคร 10600 ประเทศไทย,ตลาดพลู,ธนบุรี,2023,4,19
2023-EL9K9N,"[สัตว์จรจัด, ถนน, คนจรจัด]",เขตสวนหลวง,มีคนเร่ร่อน มาอาศัยนอนตรงข้างถนน มีสุนัขไว้เฝ้าของ สร้างกลิ่นเหม็น แถมถ้าเดินไปใกล้ๆของ ตอนเช้ามืด ต้องไปทำงาน มาเห่าจะกัดคนเดินเท้าไปมา ฝากดูแล ด้วยครีบ เป็น 3-4 ปีแล้วครับ,486 ถ. อ่อนนุช แขวงสวนหลวง กรุงเทพมหานคร 10250 ประเทศไทย,สวนหลวง,สวนหลวง,2023,8,2
876CMY,"[สัตว์จรจัด, ถนน, คนจรจัด]","เขตป้อมปราบศัตรูพ่าย,ฝ่ายเทศกิจ เขตป้อมปราบศัตรูพ่าย",ปัญหา: ประชาชนแจ้งว่า มีคนเร่ร่อนมานอน และให้อาหารสัตว์ ทั้งให้อาหารแมว สุนัข รวมถึงนก โดยเฉพาะนกเยอะมากกลัวนำเชื้อโรคเข้ามา และสกปรกมาก \nประชาชนต้องการให้เจ้าหน้าที่ตรวจสอบ\nซอย: เฉลิมเขต 1-2\nถนน: บำรุงเมือง\nจุดสังเกต: บริเวณอาคารพานิชย์ที่ร้างไม่มีคนอยู่ ในวงเวียนสวนมะลิ ถนนบำรุงเมือง เขตป้อมศัตรูพ่าย\nเขต: ป้อมปราบศัตรูพ่าย\r\n#1555,283 ซอย สวนมะลิ 1 แขวงวัดเทพศิรินทร์ เขตป้อมปราบศัตรูพ่าย กรุงเทพมหานคร 10100 ประเทศไทย,วัดเทพศิรินทร์,ป้อมปราบศัตรูพ่าย,2023,8,2
DTXU24,"[ถนน, คนจรจัด, สัตว์จรจัด]","เขตบางรัก,ฝ่ายพัฒนาชุมชนฯ เขตบางรัก",ปัญหา: พบมีคนเร่ร่อนมาอาศัยอยู่ริมถนน และเลี้ยงนกพิราบ ประชาชนเกรงว่าอาจจะเป็นแหล่งเแพร่เชื้อโรค \nถนน: มหาพฤฒาราม\nจุดสังเกต: บริเวณริมถนนดังกล่าว\nเขต: บางรัก\r\n#1555,15 ถนน มหาพฤฒาราม แขวงมหาพฤฒาราม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,มหาพฤฒาราม,บางรัก,2023,8,3
2024-E8LRCG,"[สัตว์จรจัด, ถนน, คนจรจัด]","เขตปทุมวัน,สวนหลวง (Suanluang),สำนักงานจัดการทรัพย์สิน จุฬาลงกรณ์มหาวิทยาลัย","1. (13.7420260, 100.5228615)\nพิกัดถนนบรรทัดทอง ถัดจากร้านขนมไทยแม่เดือนมา 2 ตึกแถว หรือถัดจากร้านหนึ่งนมนัวมา 3 ตึกแถว\n\n2. พบคนนำสุนัขมาขอทาน จากที่สังเกตได้เงินจากนักท่องเที่ยวต่างชาติมาบริจาคจำนวนมาก\nสังเกตในวันที่ 16 เมษายน 2567 ตั้งแต่เวลา 21.50-22.23 เป็นเวลาเกือบครึ่งชั่วโมง มีคนมาบริจาคไม่ต่ำกว่า 10 คน ส่วนใหญ่ต่างชาติให้ 20 บาท บางคนให้ถึง 200\n\nจากรูปที่ส่งจะเห็นว่าผู้หญิงเสื้อแดง เป็นเจ้าของสุนัข",1483 ถนน บรรทัดทอง แขวงวังใหม่ เขตปทุมวัน กรุงเทพมหานคร 10330 ประเทศไทย,วังใหม่,ปทุมวัน,2024,4,16
FVJUY6,"[ถนน, สัตว์จรจัด, คนจรจัด]","เขตห้วยขวาง,การรถไฟแห่งประเทศไทย",ปัญหา: ริมถนนดังกล่าว พบคนเร่ร่อนมาปลูกเพลิงพักอาศัยบนพื้นที่สาธารณะและลุกล้ำเข้าไปในที่ดินของผู้ร้อง อีกทั้งยังมีการเลี้ยงสุนัขจำนวนหลายตัว ซึ่งสุนัขดังกล่าวมีนิสัยดุร้าย มักกัดผู้คนที่สัญจรผ่านไปมา\nถนน: กำแพงเพชร 7\nจุดสังเกต: เลยวัดอุทัยธารามไปประมาณ 400 เมตร จะอยู่ทางด้านซ้ายมือ ติดกับแคมป์คนงานของบริษัทสยามชินเทค\nเขต: ห้วยขวาง\r\n#1555,4020 ถ. เพชรบุรี แขวงบางกะปิ เขตห้วยขวาง กรุงเทพมหานคร 10310 ประเทศไทย,บางกะปิ,ห้วยขวาง,2024,5,14
B8VGP4,"[ถนน, สัตว์จรจัด, คนจรจัด]","เขตสายไหม,ฝ่ายสิ่งแวดล้อมฯ เขตสายไหม",ปัญหา: บนสะพานลอยคนข้ามสะพานใหม่ (ตลาดยิ่งเจริญ) พบมีผู้นำสุนัข 2 ตัว มาผูกไว้โดยไม่มีการให้อาหารและน้ำ คาดว่ามาจากขอทานที่นำมาผูกไว้\nถนน: พหลโยธิน\nเขต: สายไหม\r\n#1555,128 ซ.แยกจากหมู่ 6 เดิม ถ. พหลโยธิน แขวงคลองถนน เขตสายไหม กรุงเทพมหานคร 10220 ประเทศไทย,คลองถนน,สายไหม,2024,9,14


In [69]:
df = df[~df['type'].apply(lambda x: set(x) == {'Unspecified'} if isinstance(x, list) else False)]
df.shape[0]


731048

In [70]:
df[['type', 'comment','organization']].sample(24)


,type,comment,organization
ticket_id,,,
AWE4YW,"[ถนน, ความปลอดภัย, แสงสว่าง]",ปัญหา: ไฟฟ้าดับที่นั่งรอรถเมล์ ทำให้มืดมากเวลามานั่งรอรถเมล์ และเกรงว่าจะเกิดอันตราย ติดตามเรื่องจากเลขอ้างอิง744P3B ยังไม่ได้รับการแก้ไข นาน5เดือนแล้ว ขอให้มาช่วยเร่งดำเนินการซ่อมเปลี่ยนหลอดไฟ\nซอย: รามคำแหง 2 ซอย 3 \nถนน: รามคำแหง2\nจุดสังเกต: ไฟส่องสว่างดับที่ป้ายรถเมล์ รามคำแหง2 ซอย3 ใกล้ปากทางเข้าโรงเรียนกรุงเทพวิทยาคม เขตประเวศ\nเขต: ประเวศ\r\n#1555,"เขตประเวศ,สำนักการจราจรและขนส่ง กรุงเทพมหานคร (สจส. กทม.)"
2023-7KB6LR,[กีดขวาง],มีรถจอดกีดขวางการจราจรเป็นทางยาวเป็นประจำ และมีอุปกรณ์ก่อสร้างตั้งทิ้งไว้นานมาก\nขอให้ผู้ที่เกี่ยวข้องช่วยดำเนินการให้ด้วย,"เขตบางพลัด,สน.บางพลัด,สำนักงานตำรวจแห่งชาติ (Royal Thai Police),กองบัญชาการตำรวจนครบาล (บช.น.),กองบังคับการตำรวจนครบาล 7 (บก.น.7)"
2022-D2BRTK,[กีดขวาง],แท็กซี่และรถตุ๊กๆ จอดขวางการจราจรทั้งสองเลน ในช่วงหลังเย็นทุกวัน ทำให้การจราจรติดขัด,"เขตวัฒนา,สน.ลุมพินี,สำนักงานตำรวจแห่งชาติ (Royal Thai Police),กองบัญชาการตำรวจนครบาล (บช.น.),กองบังคับการตำรวจนครบาล 5 (บก.น.5),ผอ.เขตวัฒนา (น.ส.สุชิรา),กลุ่มกรุงเทพใต้ (นายชาตรี วัฒนเขจร)"
UFCK47,[ถนน],ปัญหา: ริมถนนดังกล่าว ฝั่งตรงข้ามสถานีตำรวจนครบาลพหลโยธิน พบบาทวิถียุบตัวเป็นหลุมขนาดใหญ่ ปัจจุบันประชาชนตกลงไปได้รับบาดเจ็บ\nถนน: รัชดาภิเษก \nเขต: จตุจักร\r\n#1555,"เขตจตุจักร,สำนักการโยธา กทม.,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย (รฟม.),กองแผนงานและประสานสาธารณูปโภค สำนักการโยธา (กผภ.สนย.) กทม.,รองปลัดฯ นายณรงค์ เรืองศรี"
2024-HHDZU6,"[ท่อระบายน้ำ, ความปลอดภัย, ร้องเรียน]",เคยแจ้งเรื่องฝาท่อคนละระดับกับพื้นถนนไปตั้งแต่เดือนสิงหาคม 66 ทำให้คนขับขี่รถยนต์และรถจักยานยนต์เกิดความไม่สะดวก และอาจทำให้เกิดอุบัติเหตุได้ เขตแจ้งกลับว่ารองบประมาณ ตอนนี้เมษา 67 แล้วยังไม่มีวี่แววที่จะแก้ไขเลยครับ,"เขตจตุจักร,ฝ่ายโยธา เขตจตุจักร"
AV2PLC,"[ถนน, แสงสว่าง]",ปัญหา: ไฟทางดับ 2 ต้น ทำให้มืดมาก ขอให้ช่วยทำให้ไฟติด\nซอย: สวนจำปี 3\nถนน: บางบอน 3\nจุดสังเกต: เข้าซอยประมาณ 1.2 กิโลเมตร อยู่ทางขวามือ เสาเลที่ ท.14 และ ท.15\nเขต: บางแค\r\n#1555,"เขตบางแค,การไฟฟ้านครหลวง เขตบางขุนเทียน,การไฟฟ้านครหลวง MEA"
2023-3CTM4B,[กีดขวาง],นำสิ่งของมากีดขวางถนนไม่สามารถจอดรถได้,"เขตมีนบุรี,ฝ่ายเทศกิจ เขตมีนบุรี"
WY8EWD,[ถนน],ปัญหา : ถนนทางเข้าหมู่บ้านมีการปรับระดับสูงเกินไปเวลารถขับเข้าหมู่บ้านทำให้ครูดกับใต้ท้องรถ ขอให้มีการปรับปรุงเเก้ไข\nพิกัดจุดสังเกต :หมู่บ้านกลางเมื่องโยธินพัฒนา อยู่ตรงข้าม เซเว่น\nบ้านเลขที่ : ---\nถนน :---\nซอย : ประดิษฐ์มนูธรรม 19\nเเขวง :--\nเขต :ลาดพร้าว\r\n#1555,"เขตลาดพร้าว,สำนักการโยธา กทม.,ฝ่ายโยธา เขตลาดพร้าว,ผอ.เขตลาดพร้าว (นางสุภาพร)"
2022-NZFF3Z,[ท่อระบายน้ำ],ท่อหน้าบ้านยุบตัว,"เขตคลองสาน,ฝ่ายโยธา เขตคลองสาน,ผอ.เขตคลองสาน (ว่าที่ ร.ต.สรวุฒิ),กลุ่มกรุงธนเหนือ (นายเฉลิมพล โชตินุชิต)"


In [71]:
# drop if index is empty
df = df[~df.index.isna()]
int(df.index.isna().sum())


0

before split

In [72]:
df.shape[0]

724915

## Split list values into multiple rows

In [73]:
df = df.explode('type')
df


,type,organization,comment,address,subdistrict,district,year,month,day
ticket_id,,,,,,,,,
2021-FYJTFP,ความสะอาด,เขตบางซื่อ,ขยะเยอะ,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,9,3
2021-CGPMUN,น้ำท่วม,"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,2021,9,19
2021-CGPMUN,ร้องเรียน,"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,2021,9,19
2021-7XATFA,สะพาน,เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,ยานนาวา,สาทร,2021,9,26
2021-9U2NJT,น้ำท่วม,"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,บางซื่อ,บางซื่อ,2021,10,14
...,...,...,...,...,...,...,...,...,...
2025-BDQ87Y,ร้องเรียน,เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16
2025-BDQ87Y,ต้นไม้,เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16
2025-BDQ87Y,ความปลอดภัย,เขตราชเทวี,ต้นไม้บริเวณดังกล่าวช่วงโคนต้นไม้มันแห้งเหมือนต้นไม้มันตายแล้วแห้งเหมือนจะล้มลงมาเกรงว่าจะเกิดอันตรายต่อผู้สัญจรอยากให้ช่วยมาตรวจสอบแก้ไขจุดสังเกตบริเวณใกล้ปากซอยราชวิถีซอย 7,10 ถ. ราชวิถี แขวงถนนพญาไท เขตราชเทวี กรุงเทพมหานคร 10400 ประเทศไทย,ถนนพญาไท,ราชเทวี,2025,1,16


In [74]:
df.shape

(1148903, 9)

In [75]:
# Drop the 'comment' column
df.drop(columns=['comment','address','organization','district'], inplace=True)

# Check the new shape of the DataFrame
df.shape
df

,type,subdistrict,year,month,day
ticket_id,,,,,
2021-FYJTFP,ความสะอาด,บางซื่อ,2021,9,3
2021-CGPMUN,น้ำท่วม,หนองบอน,2021,9,19
2021-CGPMUN,ร้องเรียน,หนองบอน,2021,9,19
2021-7XATFA,สะพาน,ยานนาวา,2021,9,26
2021-9U2NJT,น้ำท่วม,บางซื่อ,2021,10,14
...,...,...,...,...,...
2025-BDQ87Y,ร้องเรียน,ถนนพญาไท,2025,1,16
2025-BDQ87Y,ต้นไม้,ถนนพญาไท,2025,1,16
2025-BDQ87Y,ความปลอดภัย,ถนนพญาไท,2025,1,16


In [76]:
df['date_obj'] = pd.to_datetime(df[['year', 'month', 'day']])

In [77]:
daily_data = pd.crosstab(
    index=[df['date_obj'], df['subdistrict']], 
    columns=df['type']
).reset_index()

In [78]:
target_cols = [c for c in daily_data.columns if c not in ['date_obj', 'subdistrict']]

In [79]:
daily_data['year'] = daily_data['date_obj'].dt.year
daily_data['month'] = daily_data['date_obj'].dt.month
daily_data['day'] = daily_data['date_obj'].dt.day

In [80]:

daily_data['date_obj'] = pd.to_datetime(daily_data['date_obj'])
daily_data = daily_data.sort_values(by='date_obj').reset_index(drop=True)


daily_data['month'] = daily_data['date_obj'].dt.month
daily_data['day_of_week'] = daily_data['date_obj'].dt.dayofweek
daily_data['day_of_year'] = daily_data['date_obj'].dt.dayofyear

daily_data['month_sin'] = np.sin(2 * np.pi * daily_data['month'] / 12)
daily_data['month_cos'] = np.cos(2 * np.pi * daily_data['month'] / 12)


daily_data['dow_sin'] = np.sin(2 * np.pi * daily_data['day_of_week'] / 7)
daily_data['dow_cos'] = np.cos(2 * np.pi * daily_data['day_of_week'] / 7)


daily_data['doy_sin'] = np.sin(2 * np.pi * daily_data['day_of_year'] / 365)
daily_data['doy_cos'] = np.cos(2 * np.pi * daily_data['day_of_year'] / 365)



val_start_date  = pd.Timestamp('2023-10-01')
test_start_date = pd.Timestamp('2024-01-01')


target_cols = [
    'ถนน', 'ทางเท้า', 'แสงสว่าง', 'ความสะอาด', 'กีดขวาง',
    'ท่อระบายน้ำ', 'น้ำท่วม', 'จราจร', 'สายไฟ', 'ต้นไม้',
    'สัตว์จรจัด', 'เสียงรบกวน', 'คลอง', 'PM2.5', 'ความปลอดภัย',
    'ป้าย', 'สะพาน'
]


cols_to_drop = target_cols + ['date_obj', 'month', 'day_of_week', 'day_of_year'] 



train_df = daily_data[daily_data['date_obj'] < val_start_date].copy()
val_df   = daily_data[(daily_data['date_obj'] >= val_start_date) & (daily_data['date_obj'] < test_start_date)].copy()
test_df  = daily_data[daily_data['date_obj'] >= test_start_date].copy()

X_train = train_df.drop(columns=cols_to_drop)
y_train = train_df[target_cols]

X_val = val_df.drop(columns=cols_to_drop)
y_val = val_df[target_cols]

X_test = test_df.drop(columns=cols_to_drop)
y_test = test_df[target_cols]

print(f"Features ที่ใช้เทรน: {X_train.columns.tolist()}")
print(f"Train Shape: {X_train.shape}, Val Shape: {X_val.shape}, Test Shape: {X_test.shape}")



Features ที่ใช้เทรน: ['subdistrict', 'การเดินทาง', 'คนจรจัด', 'ป้ายจราจร', 'ร้องเรียน', 'สอบถาม', 'ห้องน้ำ', 'เสนอแนะ', 'year', 'day', 'month_sin', 'month_cos', 'dow_sin', 'dow_cos', 'doy_sin', 'doy_cos']
Train Shape: (68341, 16), Val Shape: (13782, 16), Test Shape: (56696, 16)


In [81]:


model = CatBoostRegressor(
    loss_function='MultiRMSE',
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    border_count=128,
    random_seed=42,
    task_type='GPU',
    devices='0',
    thread_count=-1,
    verbose=100
)

model.fit(
    X_train, y_train,
    cat_features=['subdistrict'], 
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=200
)


0:	learn: 5.0425089	test: 4.8419378	best: 4.8419378 (0)	total: 28.7ms	remaining: 57.4s
100:	learn: 4.4740705	test: 4.5434442	best: 4.5434442 (100)	total: 2.49s	remaining: 46.8s
100:	learn: 4.4740705	test: 4.5434442	best: 4.5434442 (100)	total: 2.49s	remaining: 46.8s
200:	learn: 4.3868933	test: 4.5255188	best: 4.5255188 (200)	total: 5.01s	remaining: 44.8s
200:	learn: 4.3868933	test: 4.5255188	best: 4.5255188 (200)	total: 5.01s	remaining: 44.8s
300:	learn: 4.3255335	test: 4.5160582	best: 4.5160416 (299)	total: 7.62s	remaining: 43s
300:	learn: 4.3255335	test: 4.5160582	best: 4.5160416 (299)	total: 7.62s	remaining: 43s
400:	learn: 4.2689643	test: 4.5134503	best: 4.5134503 (400)	total: 10.3s	remaining: 41s
400:	learn: 4.2689643	test: 4.5134503	best: 4.5134503 (400)	total: 10.3s	remaining: 41s
500:	learn: 4.2264168	test: 4.5120530	best: 4.5114936 (462)	total: 12.9s	remaining: 38.7s
500:	learn: 4.2264168	test: 4.5120530	best: 4.5114936 (462)	total: 12.9s	remaining: 38.7s
600:	learn: 4.1813717

In [82]:
from sklearn.metrics import mean_absolute_error

# 1. ทำนายค่า (Predict) จากชุดข้อมูล Validation
y_pred = model.predict(X_val)

# 2. คำนวณค่า MAE ภาพรวม (Overall)
mae_overall = mean_absolute_error(y_val, y_pred)
print(f"Overall MAE: {mae_overall:.4f}")

print("-" * 30)

# 3. คำนวณค่า MAE แยกรายคอลัมน์ (เนื่องจากโมเดลทำนายหลายตัวแปรพร้อมกัน)
# ตรวจสอบว่า y_val เป็น DataFrame เพื่อดึงชื่อคอลัมน์มาแสดง
mae_per_col = mean_absolute_error(y_val, y_pred, multioutput='raw_values')

if hasattr(y_val, 'columns'):
    for i, col_name in enumerate(y_val.columns):
        print(f"MAE for {col_name}: {mae_per_col[i]:.4f}")
else:
    # กรณี y_val เป็น numpy array ไม่มีชื่อคอลัมน์
    for i, val in enumerate(mae_per_col):
        print(f"MAE for Target {i}: {val:.4f}")

Overall MAE: 0.5148
------------------------------
MAE for ถนน: 1.2948
MAE for ทางเท้า: 0.8583
MAE for แสงสว่าง: 0.7237
MAE for ความสะอาด: 0.6599
MAE for กีดขวาง: 0.6089
MAE for ท่อระบายน้ำ: 0.4391
MAE for น้ำท่วม: 0.3919
MAE for จราจร: 0.4521
MAE for สายไฟ: 0.2576
MAE for ต้นไม้: 0.3718
MAE for สัตว์จรจัด: 0.2726
MAE for เสียงรบกวน: 0.4365
MAE for คลอง: 0.3101
MAE for PM2.5: 0.1602
MAE for ความปลอดภัย: 0.6562
MAE for ป้าย: 0.5123
MAE for สะพาน: 0.3456


In [83]:
import pickle

# Save the trained model
model.save_model('bangkok_traffy_model.cbm')

# Save feature names and target columns for API
model_metadata = {
    'feature_names': X_train.columns.tolist(),
    'target_cols': target_cols,
    'subdistricts': X_train['subdistrict'].unique().tolist()
}

with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(model_metadata, f)

print("✓ Model saved as 'bangkok_traffy_model.cbm'")
print("✓ Metadata saved as 'model_metadata.pkl'")
print(f"✓ Features: {len(model_metadata['feature_names'])}")
print(f"✓ Target categories: {len(target_cols)}")
print(f"✓ Subdistricts: {len(model_metadata['subdistricts'])}")

✓ Model saved as 'bangkok_traffy_model.cbm'
✓ Metadata saved as 'model_metadata.pkl'
✓ Features: 16
✓ Target categories: 17
✓ Subdistricts: 178
